In [10]:
import sys, subprocess, time, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_aer.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.optimizers import COBYLA

In [11]:
# ===== Parametry do łatwej zmiany =====
data_path   = "countsAll_fixed_07_07_23.csv"  # ścieżka do pliku z danymi
sep         = "\t"                            # separator (w Twoim pliku jest tab)
n_components_pca = 4                          # liczba komponentów PCA = liczba kubitów
test_size   = 0.20                            # ułamek danych do testu
random_state = 42                             # ziarno losowe
maxiter     = 200                             # iteracje optymalizatora
entanglement = "linear"                       # "linear" | "full" | lista par
reps_feature = 2                              # głębokość feature map
reps_ansatz  = 2  

In [12]:
# ===== 1) Wczytanie i przygotowanie danych =====
print("Wczytywanie danych...")
df = pd.read_csv(data_path, sep=sep)

Wczytywanie danych...


In [13]:
def infer_labels_from_columns(columns):
    """
    Etykieta 0: próbki zdrowe (zawierają '-HD-' w nazwie),
    Etykieta 1: próbki z RNA nowotworowym (pozostałe).
    """
    return np.array([0 if "-HD-" in c else 1 for c in columns], dtype=int)

In [14]:
# Wiersze = geny, kolumny = próbki; transpozycja => próbki x geny
X = df.T
y = infer_labels_from_columns(X.index.tolist())

print(f"Liczba cech (genów): {X.shape[1]}, liczba próbek: {X.shape[0]}")
print(f"Klasy: 0 (zdrowe) = {(y==0).sum()}, 1 (nowotworowe) = {(y==1).sum()}")

Liczba cech (genów): 5346, liczba próbek: 2351
Klasy: 0 (zdrowe) = 280, 1 (nowotworowe) = 2071


In [15]:
# ===== 2) Podział na zbiory uczący i testowy =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y
)

In [16]:
# ===== 3) Standaryzacja + PCA (dopasowana na zbiorze uczącym) =====
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std  = scaler.transform(X_test)

pca = PCA(n_components=n_components_pca, random_state=random_state)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca  = pca.transform(X_test_std)

print(f"Po PCA: X_train = {X_train_pca.shape}, X_test = {X_test_pca.shape}")

Po PCA: X_train = (1880, 4), X_test = (471, 4)


In [20]:
# ===== 4) Budowa VQC =====
feature_map = ZZFeatureMap(feature_dimension=n_components_pca, reps=reps_feature)
ansatz      = TwoLocal(
    n_components_pca,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cz",
    entanglement=entanglement,
    reps=reps_ansatz,
)

optimizer = COBYLA(maxiter=maxiter)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
)

In [ ]:
# ===== 5) Trening =====
print("\nTrening VQC...")
vqc.fit(X_train_pca, y_train)


Trening VQC...


In [ ]:
# ===== 6) Ewaluacja =====
y_pred = vqc.predict(X_test_pca)
acc = accuracy_score(y_test, y_pred)

print("\n=== Wyniki ===")
print(f"Dokładność na zbiorze testowym: {acc:.4f}")